# Data 
https://www.kaggle.com/competitions/rossmann-store-sales/overview

# Import

## Import Library

In [19]:
# ライブラリーの読み込み
import numpy as np
import pandas as pd
import pmdarima as pm
import seaborn as sns
from pmdarima import datasets
from pmdarima import utils
from pmdarima import arima
from pmdarima import model_selection
from sklearn.metrics import mean_absolute_error
from statistics import mean 
from matplotlib import pyplot as plt
# グラフのスタイルとサイズ
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [12, 9]

from statsmodels.tsa.arima.model import ARIMA

ModuleNotFoundError: No module named 'seaborn'

## Import Data

In [2]:
!ls data/rossman-store-sales/

sample_submission.csv  store.csv  test.csv  train.csv


In [3]:
# データの読み込み
df_train = pd.read_csv('./data/rossman-store-sales/train.csv', encoding='utf-8')
df_test = pd.read_csv('./data/rossman-store-sales/test.csv')
df_store = pd.read_csv('./data/rossman-store-sales/store.csv')

<ipython-input-3-4d2c62b36edd>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./data/rossman-store-sales/train.csv', encoding='utf-8')


In [4]:
print('***** Check Columns *****')
print('train:', df_train.columns.to_list())
print('test :', df_test.columns.to_list())

print('***** Check Length *****')
print('train:', len(df_train))
print('test :', len(df_test))

***** Check Columns *****
train: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
test : ['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
***** Check Length *****
train: 1017209
test : 41088


# データ加工

## 中身確認

In [5]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [15]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [18]:
df_store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [17]:
df_store.groupby(['StoreType']).count()

,Store,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
StoreType,,,,,,,,,
a,602,602,601,426,426,602,289,289,289
b,17,17,17,9,9,17,5,5,5
c,148,148,148,107,107,148,75,75,75
d,348,348,346,219,219,348,202,202,202


## 必要な情報だけ抽出

In [7]:
# df_testのIdがいらないので削除
df_test = df_test.drop('Id',axis=1)
df_test.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,4,2015-09-17,1.0,1,0,0
1,3,4,2015-09-17,1.0,1,0,0
2,7,4,2015-09-17,1.0,1,0,0
3,8,4,2015-09-17,1.0,1,0,0
4,9,4,2015-09-17,1.0,1,0,0


In [8]:
print(df_train[df_train['Open']==0])
print(df_train[df_train['Open']==1])

         Store  DayOfWeek        Date  Sales  Customers  Open  Promo  \
291        292          5  2015-07-31      0          0     0      1   
875        876          5  2015-07-31      0          0     0      1   
1406       292          4  2015-07-30      0          0     0      1   
1990       876          4  2015-07-30      0          0     0      1   
2521       292          3  2015-07-29      0          0     0      1   
...        ...        ...         ...    ...        ...   ...    ...   
1017204   1111          2  2013-01-01      0          0     0      0   
1017205   1112          2  2013-01-01      0          0     0      0   
1017206   1113          2  2013-01-01      0          0     0      0   
1017207   1114          2  2013-01-01      0          0     0      0   
1017208   1115          2  2013-01-01      0          0     0      0   

        StateHoliday  SchoolHoliday  
291                0              1  
875                0              1  
1406               0 

In [9]:
print(len(df_train[df_train['Open']==0]))
print(len(df_train[df_train['Open']==1]))

df_train = df_train[df_train['Open']==1]

172817
844392


## df_storeをmerge

In [14]:
df_train_all = pd.merge(df_train, df_store, how='inner', on="Store")
df_train_all.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


# データの確認

# 売上を全部足しあげて時系列分析 

In [ ]:
df_sum = df_train.groupby(by=['Date'])['Sales'].agg('sum')

In [ ]:
df_sum

In [ ]:
df_sum_pd = pd.DataFrame(df_sum)#.transpose()

In [ ]:
df_sum_pd.columns

In [ ]:
df_sum_train = df_sum_pd[:-31]
df_sum_test  = df_sum_pd[-31:]

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(df_sum_test['Sales'])
plt.xlabel("Date", fontsize=14)
plt.ylabel("Sales", fontsize=14)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
df_sum_train = df_sum[df_sum.index >= "2014-07-01"]
df_sum_test = df_sum[df_sum.index >= "2015-07-01"]


In [ ]:
plt.figure(figsize=(60, 8))
plt.plot(df_sum_train)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Sales", fontsize=14)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(df_sum_test)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Sales", fontsize=14)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
utils.decomposed_plot(arima.decompose(df_sum_train, 'additive', m=31),
                     figure_kwargs = {'figsize':(12,12)})

In [ ]:
df_sum = df_sum[df_sum.index >= "2014-07-01"]
print(len(df_sum))

In [ ]:
# 階差の次数の検討
print('d =', arima.ndiffs(df_sum)) #d（階差）
print('D =', arima.nsdiffs(df_sum,m=31)) #D（季節階差）


In [ ]:
# 通常の階差処理を1回しさらに季節階差処理を1回
data_d1 = pd.DataFrame(df_sum).diff(1).dropna()

In [ ]:
# データ分割（train:学習データ、test:テストデータ）
train, test = model_selection.train_test_split(df_sum, test_size=31)

In [ ]:
# モデル構築（Auto ARIMA）
arima_model = pm.auto_arima(train, 
                            seasonal=True,
                            m=31,
                            trace=True,
                            n_jobs=-1,
                            maxiter=10)

In [ ]:
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 9]
# 予測
preds, conf_int = arima_model.predict(n_periods=test.shape[0], 
                                      return_conf_int=True)
# 予測精度
print('MAE:')
print(mean_absolute_error(test, preds)) 
print('MAPE(%):')
print(mean(abs(test - preds)/test) *100)
# 予測と実測の比較（グラフ）
x_axis = np.arange(preds.shape[0])
plt.plot(x_axis,test,label="actual",color='r') 
plt.plot(x_axis,preds,label="predicted",color='b')
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b')
plt.legend()
plt.show()
 

In [ ]:
# グラフ（学習データとテストデータ、予測結果）
x_axis = np.arange(train.shape[0] + preds.shape[0])
plt.plot(x_axis[:train.shape[0]],train,color='r',label="actual")
plt.plot(x_axis[train.shape[0]:],test,color='r')
plt.plot(x_axis[train.shape[0]:],preds,color='b',label="predicted")
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1,color='b')
plt.legend()
plt.show()
 

In [ ]:
# モデル構築（Auto ARIMA）
arima_model = pm.auto_arima(train, 
                             seasonal=True,
                             m=31,
                             d=1,
#                              D=1,
                             n_jobs=-1,
                             maxiter=10)
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 9]
# 予測
preds, conf_int = arima_model.predict(n_periods=test.shape[0], 
                                      return_conf_int=True)
# 予測精度
print('MAE:')
print(mean_absolute_error(test, preds)) 
print('MAPE(%):')
print(mean(abs(test - preds)/test) *100)
# 予測と実測の比較（グラフ）
x_axis = np.arange(preds.shape[0])
plt.plot(x_axis,test,label="actual",color='r') 
plt.plot(x_axis,preds,label="predicted",color='b')
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b')
plt.legend()
plt.show()
# グラフ（学習データとテストデータ、予測結果）
x_axis = np.arange(train.shape[0] + preds.shape[0])
plt.plot(x_axis[:train.shape[0]],train,color='r',label="actual")
plt.plot(x_axis[train.shape[0]:],test,color='r')
plt.plot(x_axis[train.shape[0]:],preds,color='b',label="predicted")
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1,color='b')
plt.legend()
plt.show()

# 売上が一番大きい店舗について、注目して分析

In [ ]:
df_sum_store = df_train.groupby(by=['Store'])['Sales'].agg('sum')

In [ ]:
df_sum_store.sort_values()
# -> 262

In [ ]:
df_train_262 = df_train[df_train['Store']==262]

In [ ]:
df_train_262

In [ ]:
df_train_262_sale = pd.Series(list(df_train_262['Sales']), index=df_train_262['Date'])

In [ ]:
df_train_262_sale

In [ ]:
df_train_262_sale_train = df_train_262_sale[df_train_262_sale.index >= "2014-07-01"]
print(len(df_train_262_sale_train))

In [ ]:
plt.figure(figsize=(60, 8))
plt.plot(df_train_262_sale)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Sales", fontsize=14)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
utils.decomposed_plot(arima.decompose(df_train_262_sale, 'additive', m=31),
                     figure_kwargs = {'figsize':(12,12)})

In [ ]:
# 階差の次数の検討
print('d =', arima.ndiffs(df_train_262_sale)) #d（階差）
print('D =', arima.nsdiffs(df_train_262_sale,m=365)) #D（季節階差）

In [ ]:
# ADF検定（原系列）
dftest = adfuller(df_train_262_sale)
print('ADF Statistic: %f' % dftest[0])
print('p-value: %f' % dftest[1])
print('Critical values :')
for k, v in dftest[4].items():
    print('\t', k, v)

In [ ]:
utils.decomposed_plot(arima.decompose(np.log(df_train_262_sale), 'additive', m=31),
                     figure_kwargs = {'figsize':(12,12)})

In [ ]:
# ADF検定（原系列）
dftest = adfuller(np.log(df_train_262_sale))
print('ADF Statistic: %f' % dftest[0])
print('p-value: %f' % dftest[1])
print('Critical values :')
for k, v in dftest[4].items():
    print('\t', k, v)

In [ ]:
# 階差
df_log_diff1 = df_train_262_sale.diff(1).dropna()
plt.plot(df_log_diff1)

In [ ]:
# ADF検定
dftest = adfuller(df_log_diff1)
print('ADF Statistic: %f' % dftest[0])
print('p-value: %f' % dftest[1])
print('Critical values :')
for k, v in dftest[4].items():
    print('\t', k, v)

In [ ]:
# from statsmodels.tsa.stattools import adfuller


# def my_adf(d):
#     ps = [0.01,0.05]
#     for reg in ['nc','c','ct','ctt']:
#         adf,pvalue,usedlag,nobs,critical_values,icbest = adfuller(
#             x = d,
#             regression = reg,
#             autolag = 'AIC'
#         )
#     ps.append(pvalue)

#     return ps

# fig,axs = plt.subplots(4,1)
# #a1=1のケースの階差をとる
# # data4 = pd.Series(data2).diff().dropna()
# for ax,d in zip(axs,df_train_262_sale_train):
#     ax.bar(height=my_adf(d),x='0.01,0.05,nc,c,ct,ctt'.split(','))

In [ ]:
# データ分割（train:学習データ、test:テストデータ）
train, test = model_selection.train_test_split(df_train_262_sale, test_size=31)

# モデル構築（Auto ARIMA）
arima_model = pm.auto_arima(train, 
                             seasonal=False,
                             m=365,
#                              d=1,
#                              D=1,
                             n_jobs=-1,
                             maxiter=10)
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 9]
# 予測
preds, conf_int = arima_model.predict(n_periods=test.shape[0], 
                                      return_conf_int=True)
# 予測精度
print('MAE:')
print(mean_absolute_error(test, preds)) 
print('MAPE(%):')
print(mean(abs(test - preds)/test) *100)
# 予測と実測の比較（グラフ）
x_axis = np.arange(preds.shape[0])
plt.plot(x_axis,test,label="actual",color='r') 
plt.plot(x_axis,preds,label="predicted",color='b')
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b')
plt.legend()
plt.show()
# グラフ（学習データとテストデータ、予測結果）
x_axis = np.arange(train.shape[0] + preds.shape[0])
plt.plot(x_axis[:train.shape[0]],train,color='r',label="actual")
plt.plot(x_axis[train.shape[0]:],test,color='r')
plt.plot(x_axis[train.shape[0]:],preds,color='b',label="predicted")
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1,color='b')
plt.legend()
plt.show()

## Add Columns : Month

In [ ]:
df_sum_month = df_train.groupby(by=['Month'])['Sales'].agg('sum')

In [ ]:
df_sum_month

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(df_sum_month)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Sales", fontsize=14)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
utils.decomposed_plot(arima.decompose(df_sum_month, 'additive', m=12),
                     figure_kwargs = {'figsize':(12,12)})

In [ ]:
plt.plot(df['Value (kWh)'])

In [ ]:
model = ARIMA(df['Value (kWh)'], order=(1,1,1))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Plotting residual errors
residuals = pd.DataFrame(model_fit.resid)
plt.figure(figsize=(18, 8))
plt.plot(residuals)
plt.title('Residuals', fontsize=16)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Amount of residuals", fontsize=14)
plt.xticks(np.arange(0, len(df.StartDate)+1, 45), labels=[df.StartDate[i] for i in range(0, len(df.StartDate)+1, 45)], rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
train = df['meantemp'][:int(np.floor((len(df)/100)*70))]
test = df['meantemp'][int(np.floor((len(df)/100)*70)):]

model = ARIMA(train, order=(3, 2, 1))  
fitted = model.fit()  

fc = fitted.get_forecast(len(df['meantemp'][int(np.floor((len(df)/100)*70)):]))  
conf = fc.conf_int(alpha=0.05) # 95% confidence

fc_series = pd.Series(fc.predicted_mean, index=test.index)
lower_series = pd.Series(conf.iloc[:, 0], index=test.index)
upper_series = pd.Series(conf.iloc[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=200)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()


In [ ]:
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 3]
# 自己相関と偏自己相関
utils.plot_acf(df.meantemp, alpha=.05)
utils.plot_pacf(df.meantemp, alpha=.05)
 

In [ ]:
# 階差の次数の検討
print('d =', arima.ndiffs(data)) #d（階差）
print('D =', arima.nsdiffs(data,m=12)) #D（季節階差）


In [ ]:
# 通常の階差処理を1回しさらに季節階差処理を1回
data_d1_d12 = pd.DataFrame(data).diff(1).diff(12).dropna()

In [ ]:
data_d1_d12

In [ ]:
# データ分割（train:学習データ、test:テストデータ）
train, test = model_selection.train_test_split(data, train_size=132)

In [ ]:
# モデル構築（Auto ARIMA）
arima_model = pm.auto_arima(train, 
                            seasonal=True,
                            m=12,
                            trace=True,
                            n_jobs=-1,
                            maxiter=10)

In [ ]:
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 9]
# 予測
preds, conf_int = arima_model.predict(n_periods=test.shape[0], 
                                      return_conf_int=True)
# 予測精度
print('MAE:')
print(mean_absolute_error(test, preds)) 
print('MAPE(%):')
print(mean(abs(test - preds)/test) *100)
# 予測と実測の比較（グラフ）
x_axis = np.arange(preds.shape[0])
plt.plot(x_axis,test,label="actual",color='r') 
plt.plot(x_axis,preds,label="predicted",color='b')
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b')
plt.legend()
plt.show()
 

In [ ]:
# グラフ（学習データとテストデータ、予測結果）
x_axis = np.arange(train.shape[0] + preds.shape[0])
plt.plot(x_axis[:train.shape[0]],train,color='r',label="actual")
plt.plot(x_axis[train.shape[0]:],test,color='r')
plt.plot(x_axis[train.shape[0]:],preds,color='b',label="predicted")
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1,color='b')
plt.legend()
plt.show()
 

In [ ]:
# モデル構築（Auto ARIMA）
arima_model = pm.auto_arima(train, 
                             seasonal=True,
                             m=12,
                             d=1,
                             D=1,
                             n_jobs=-1,
                             maxiter=10)
# グラフのサイズ変更
plt.rcParams['figure.figsize'] = [12, 9]
# 予測
preds, conf_int = arima_model.predict(n_periods=test.shape[0], 
                                      return_conf_int=True)
# 予測精度
print('MAE:')
print(mean_absolute_error(test, preds)) 
print('MAPE(%):')
print(mean(abs(test - preds)/test) *100)
# 予測と実測の比較（グラフ）
x_axis = np.arange(preds.shape[0])
plt.plot(x_axis,test,label="actual",color='r') 
plt.plot(x_axis,preds,label="predicted",color='b')
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b')
plt.legend()
plt.show()
# グラフ（学習データとテストデータ、予測結果）
x_axis = np.arange(train.shape[0] + preds.shape[0])
plt.plot(x_axis[:train.shape[0]],train,color='r',label="actual")
plt.plot(x_axis[train.shape[0]:],test,color='r')
plt.plot(x_axis[train.shape[0]:],preds,color='b',label="predicted")
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1,color='b')
plt.legend()
plt.show()